In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from kaggle_secrets import UserSecretsClient



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/esc-rag/Dataset/Agarwal and Lang 2005 Foundations of Analog and Digital.txt
/kaggle/input/esc-rag/Dataset/Hayt Engineering Circuit Analysis 8th txtbk.txt
/kaggle/input/esc-rag/Dataset/Microelectronic Circuits 7ed 2015.txt
/kaggle/input/esc-rag/Dataset/DIGITAL PRINCIPLES AND APPLICATION BY LEACH MALVINO.txt
/kaggle/input/esc-rag/Dataset/Electronic Devices and Circuit Theory.txt
/kaggle/input/esc-rag/Dataset/Digital Design 4th Edition - Morris Mano.txt
/kaggle/input/esc-data/Dataset/DIGITAL PRINCIPLES AND APPLICATION BY LEACH  MALVINO.pdf
/kaggle/input/esc-data/Dataset/Hayt Engineering Circuit Analysis 8th txtbk.pdf
/kaggle/input/esc-data/Dataset/Microelectronic Circuits 7ed 2015.pdf
/kaggle/input/esc-data/Dataset/Electronic Devices and Circuit Theory.pdf
/kaggle/input/esc-data/Dataset/Agarwal and Lang 2005 Foundations of Analog and Digital.pdf
/kaggle/input/esc-data/Dataset/Digital Design 4th Edition - Morris Mano.pdf


In [2]:
import os
from kaggle_secrets import UserSecretsClient

# Retrieve the token from the environment variable
secret_label = "HF_TOKEN"
hf_token = UserSecretsClient().get_secret(secret_label)
from huggingface_hub import login
login(token = hf_token,add_to_git_credential=True)

Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id , use_auth_token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token=hf_token,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    
)
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)


/opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
!pip install pinecone
!pip install --quiet langchain_experimental langchain_openai
! pip install langchain==0.1.1 pinecone-client==2.2.4 python-dotenv==1.0.0 streamlit==1.29.0
!pip install sentence-transformers
!pip install langchain_pinecone
!pip install protoc_gen_openapiv2

  Using cached langchain-0.1.1-py3-none-any.whl.metadata (13 kB)
  Using cached pinecone_client-2.2.4-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_community-0.0.37-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_community-0.0.36-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_community-0.0.35-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_community-0.0.34-py3-none-any.whl.metadata (8.5 kB)
  Using cached langchain_community-0.0.33-py3-none-any.whl.metadata (8.5 kB)
  Using cached langchain_community-0.0.32-py3-none-any.whl.metadata (8.5 kB)
  Using cached langchain_comm

In [5]:

# # Directory containing your .txt files
# directory = '/kaggle/input/esc-rag/Dataset'  # Replace with your directory path

# # Output file name
# output_file = 'concatenated_books.txt'

# # Open the output file in write mode
# with open(output_file, 'w', encoding='utf-8') as outfile:
#     # Iterate through all .txt files in the directory
#     for filename in sorted(os.listdir(directory)):
#         if filename.endswith('.pdf'):
#             file_path = os.path.join(directory, filename)
#             with open(file_path, 'r', encoding='utf-8') as infile:
#                 # Read the content of each file
#                 content = infile.read()
#                 # Optionally, add a separator or filename to the output
#                 outfile.write(f"\n--- Start of {filename} ---\n\n")
#                 outfile.write(content)
#                 outfile.write(f"\n--- End of {filename} ---\n\n")

# print(f"All files have been concatenated into {output_file}")


In [6]:
# This is a long document we can split up.
with open("/kaggle/working/concatenated_books.txt") as f:
    concatenated_books = f.read()

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings

loader = TextLoader('/kaggle/working/concatenated_books.txt')
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=4)
docs = text_splitter.split_documents(documents)


In [8]:
print(len(docs))
print(docs[1100].page_content)

14924
Because the input small-signal voltage is set to zero, the current through
the MOSFET is 0. In other words, the MOSFET behaves like an open circuit.
Thus the output resistance for small signals is RL .
Current Gain
Analogous to the voltage gain, we can define a current gain for an amplifier
that supplies an external current.
Incremental current gain The change in the output current divided by the
change in the input current, for a given external load resistance.
As depicted in Figure 8.22, we can compute the current gain by applying a
small test voltage at the input and measuring both the input current itest and
the output current io . The ratio io /itest is the current gain. Note that the output
current is not the current that flows through the dependent current source,
rather it is the current that is drawn by an external load resister RO . Because
it is dependent on the value of the load resistor, the current gain is defined

426

CHAPTER EIGHT

the small-signal model


In [9]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

2024-08-17 06:10:10.601619: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 06:10:10.601692: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-17 06:10:10.605802: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:


# from langchain_pinecone import PineconeEmbeddings

# # model_name = "multilingual-e5-large"  
# # embeddings = PineconeEmbeddings(  
# #     model=model_name,  
# #     pinecone_api_key=UserSecretsClient().get_secret("PINECONE_API_KEY") 
# # )  


In [11]:
os.environ["PINECONE_API_KEY"] = UserSecretsClient().get_secret('PINECONE_API_KEY')


In [21]:
from langchain.vectorstores import Pinecone
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from langchain_pinecone import PineconeVectorStore
# Initialize Pinecone client
pc=Pinecone(
    api_key= UserSecretsClient().get_secret('PINECONE_API_KEY'),
    
)

# Define Index Name
index_name = "rag-esc"



if index_name not in pc.list_indexes().names():
    pc.create_index(
        name="rag-esc",
        dimension=768, 
        metric="cosine", 
        spec=ServerlessSpec(
            cloud="aws", 
            region="us-east-1"
        ) 
    )
    docsearch = PineconeVectorStore.from_documents(
        documents=docs,
        index_name="rag-esc",
        embedding=embeddings, 
    )
    
else:
    from langchain.vectorstores import Pinecone
    docsearch = Pinecone.from_existing_index(index_name, embeddings)

In [23]:
index = pc.Index(index_name)

In [ ]:
# query="Thevenin and norton"

# # result = index.query(vector=embeddings.embed_query(query), top_k=20, include_metadata=True)

# # # Display results
# # for match in result['matches']:
# #     print(f"ID: {match['id']}, Score: {match['score']},Text:{match['metadata']}")

In [34]:
# Initialize the template
def format_documents(query):
    results = index.query(vector=embeddings.embed_query(query), top_k=5, include_metadata=True)
    formatted_docs = []
    for match in results['matches']:
        doc_id = match['id']
        score = match['score']
        text = match['metadata'].get('text', 'No text available')  # Assuming metadata has a 'text' field
        
        # Create a formatted template
        formatted_doc = f"""
        Document ID: {doc_id}
        Relevance Score: {score}

        Content:
        {text}
        """
        
        formatted_docs.append(formatted_doc)
        
    prompt_template=f'''
    User={query}
    Suggested_docs={formatted_docs}
    Assistant: '''
    
    # Join all formatted documents into a single input string
#     return "\n\n---\n\n".join(formatted_docs)
    return prompt_template


In [31]:


def llama3(formatted_doc):
#     prompt = query
#     relevant
    
    messages = [
      {"role": "system", "content": " You are a helpful assistant and you have to generate a correct answer for the given User query."},
      {"role": "user", "content": formatted_doc},
  ]

    input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

    terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

    outputs = model.generate(
      input_ids,
      max_new_tokens=1024,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )
    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)
  
     


In [36]:
query="What is p-n juntion diode"


In [37]:
def answer(query):
    formatted_doc=format_documents(query)
    answer=llama3(formatted_doc)
    print(answer)

In [ ]:
answer("What is avalanche diode")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Based on the provided documents, an Avalanche Diode is a type of diode that exhibits a sharp increase in current when the voltage across it increases in the reverse-bias region. This increase in current is due to an ionization process where minority carriers gain sufficient energy to release additional carriers through collisions with atomic structures, resulting in a high avalanche current.

In the context of a p-n junction, the avalanche region is the area where the voltage across the diode increases in the reverse-bias region, leading to a rapid increase in current. This region is characterized by a high velocity and associated kinetic energy of minority carriers, which can release additional carriers through collisions with atomic structures.

Avalanche diodes are often used in applications where a high current is required in a short period, such as in power supplies, motor control, and switching circuits. They are also used in Zener diodes, which are designed to operate in the ava